In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
from tqdm import tqdm


from model.SpeechGenerator import SpeechGenerator
from utils.Config import ConfigSLP, ConfigNAC, ConfigDiTTO
from utils.MLS import MLSDataset
from utils.Trainer import Trainer
from utils.Processing import Processing

from torch.utils.data import DataLoader

In [16]:
ConfigSLP.display()
ConfigNAC.display()
ConfigDiTTO.display()

############# Base Settings: #############

Audio Settings:
  Sample Rate: 24000 Hz
  Min Audio Duration: 10 seconds
  Max Audio Duration: 20 seconds

Training Settings:
  Betas: [0.9, 0.999]
  Device: cuda

Data Settings:
  Train Path: /tempory/M2-DAC/UE_DEEP/AMAL/DiTTO-TTS/data/mls_french_opus/train
  Test Path: /tempory/M2-DAC/UE_DEEP/AMAL/DiTTO-TTS/data/mls_french_opus/test
  Dev Path: /tempory/M2-DAC/UE_DEEP/AMAL/DiTTO-TTS/data/mls_french_opus/dev

############# SLP Settings: #############

Model Settings:
  Model Name: SLP
  Embedding Dimension: 1472
  Number of Layers: 1
  Number of Attention Heads: 1
  Number of Classes: 11
  Number of Samples: 10000
  Batch Size: 8
  Learning Rate: 0.0001
  Epochs: 20
  Token length for ByT5: 128
############# Base Settings: #############

Audio Settings:
  Sample Rate: 24000 Hz
  Min Audio Duration: 10 seconds
  Max Audio Duration: 20 seconds

Training Settings:
  Betas: [0.9, 0.999]
  Device: cuda

Data Settings:
  Train Path: /tempory/M2-DA

In [17]:
# Processing.remove_metadata_from_audio_folder(ConfigSLP.TRAIN_PATH+"/"+"audio", ConfigSLP.TRAIN_PATH+"/"+"audio_clean",)
# Processing.remove_metadata_from_audio_folder(ConfigSLP.TEST_PATH+"/"+"audio", ConfigSLP.TEST_PATH+"/"+"audio_clean",)
# Processing.remove_metadata_from_audio_folder(ConfigSLP.DEV_PATH+"/"+"audio", ConfigSLP.DEV_PATH+"/"+"audio_clean",)

## Speech Generation with DiTTO-TTs and Vocoder

In [18]:
speech_generator = SpeechGenerator(
    nac_model_path="/tempory/M2-DAC/UE_DEEP/AMAL/DiTTO-TTS/src/params/NAC_epoch_20.pth",
    ditto_model_path="/tempory/M2-DAC/UE_DEEP/AMAL/DiTTO-TTS/src/params/DiTTO_epoch_20.pth",
    lambda_factor=ConfigNAC.LAMBDA_FACTOR,
    sample_rate=ConfigNAC.SAMPLE_RATE,
)

[INFO] Loading NAC model...


/tempory/M2-DAC/UE_DEEP/AMAL/DiTTO-TTS/.venv/lib/python3.9/site-packages/transformers/models/encodec/modeling_encodec.py:124: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("padding_total", torch.tensor(kernel_size - stride, dtype=torch.int64), persistent=False)
/tempory/M2-DAC/UE_DEEP/AMAL/DiTTO-TTS/src/model/DiTTO.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbi

[INFO] NAC Loaded.


/tempory/M2-DAC/UE_DEEP/AMAL/DiTTO-TTS/src/model/SpeechGenerator.py:40: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ditto_info = torch.load(ditto_model_path, map_location=

Loading weights from nvidia/bigvgan_v2_24khz_100band_256x


/tempory/M2-DAC/UE_DEEP/AMAL/DiTTO-TTS/src/bigvgan_v2_24khz_100band_256x/bigvgan.py:481: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_dict = torch.load(model_fil

Removing weight norm...


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
speech_generator.generate_speech_from_audio_tensor(
    
)